### Homework 2020/11/01, Roger Wang

# Mission 1
請至環保署空品網下載個人所在地區2019 年逐時資料
挑選其最近的觀測站讀入 列出 監測記錄總天數 與 測項總項數

In [1]:
import pandas as pd
import numpy as np

def data_clean(tdf):
    new_col = dict(zip(tdf.columns, tdf.columns.str.strip(' '))) 
    tdf.rename(columns=new_col, inplace=True)

    for col in df.columns:
        tdf[col] = tdf[col].str.replace('NA','0')
        if col == '測項':
            tdf[col] = tdf[col].str.strip(' \n')
        if (col != '測站') and (col != '日期') and (col != '測項'):
            tdf[col] = tdf[col].str.strip(' *#ANx')
            tdf[col] = tdf[col].astype('float', copy=False, errors='ignore')
    return tdf
    
df = pd.read_csv('湖口_2019.csv', encoding='big5')

df = data_clean(df)

gp = df[['測項']].groupby(by=df['日期'])
display(gp.count())
print('監測記錄總天數:{}天'.format(len(list(gp))))
print('測項總項數:{}項'.format(gp.count().iloc[0,0]))

,測項
日期,
2019/01/01 00:00:00,15
2019/01/02 00:00:00,15
2019/01/03 00:00:00,15
2019/01/04 00:00:00,15
2019/01/05 00:00:00,15
...,...
2019/12/27 00:00:00,15
2019/12/28 00:00:00,15
2019/12/29 00:00:00,15


監測記錄總天數:365天
測項總項數:15項


## Mission 2
<img style="float: left;" src="mission2.png" width="30%"><br>
<br>
請將讀入資料集欄位
(Column)轉換為下列型態：
西元年(YYYY)、月(MM)、日(DD)、時(HH)、觀測站及各測項類別

溫度:AMB_TEMP(℃),一氧化碳:CO(ppm),一氧化氮:NO(ppb),二氧化氮:NO2(ppb),氮氧化物:NOx(ppb),臭氧:O3(ppb)
懸浮微粒:PM10(μg/m3),細懸浮微粒:PM2.5(μg/m3),降雨量:RAINFALL(mm/h),相對濕度:RH(percent),二氧化硫:SO2(ppb)
小時風向值:WD_HR(degrees),風向：WIND_DIREC,風速:WIND_SPEED(m/sec),小時風速值:WS_HR(m/sec)

In [2]:
df1 = df.copy()
hr_cols=[ '00','01','02','03','04','05','06','07','08','09',
          '10','11','12','13','14','15','16','17','18','19',
          '20','21','22','23']
gp1 = df1.groupby('測項')
df_transform = pd.DataFrame()
for i in gp1:
    station = i[0]
    df_temp = i[1]
    for j in range(len(df_temp)):
        for hr in range(0,24):
            idx = j*24+hr
            if(df_transform.columns.shape[0]<df_temp.shape[0]):
                df_transform.loc[idx,'西元年'] = df_temp.iloc[j,1][:10].split('/')[0]
                df_transform.loc[idx,'月'] = df_temp.iloc[j,1][:10].split('/')[1]
                df_transform.loc[idx,'日'] = df_temp.iloc[j,1][:10].split('/')[2]
                df_transform.loc[idx,'時'] = hr_cols[hr]
                df_transform.loc[idx,'觀測站'] = df_temp.iloc[j,0]
            df_transform.loc[idx, station] = df_temp.iloc[j, 3+hr]
    
display(df_transform)

,西元年,月,日,時,觀測站,AMB_TEMP,CO,NO,NO2,NOx,O3,PM10,PM2.5,RAINFALL,RH,SO2,WD_HR,WIND_DIREC,WIND_SPEED,WS_HR
0,2019,01,01,00,湖口,15.3,0.17,0.7,4.4,5.1,35.8,14.0,8.0,0.0,84.0,1.3,51.0,45.0,6.8,6.6
1,2019,01,01,01,湖口,15.2,0.17,1.7,4.6,6.3,36.0,10.0,9.0,0.0,84.0,1.2,48.0,47.0,7.1,6.5
2,2019,01,01,02,湖口,15.3,0.16,1.6,3.7,5.4,37.7,12.0,12.0,0.0,84.0,1.2,47.0,48.0,6.9,6.7
3,2019,01,01,03,湖口,15.2,0.16,1.7,3.4,5.1,39.1,12.0,10.0,0.0,84.0,1.2,46.0,48.0,6.7,6.4
4,2019,01,01,04,湖口,15.2,0.15,1.8,3.2,5.0,40.1,5.0,6.0,0.0,84.0,1.5,45.0,46.0,6.8,6.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,2019,12,31,19,湖口,13.2,0.28,0.3,6.8,7.2,35.7,22.0,13.0,0.0,83.0,1.4,46.0,49.0,8.8,6.7
8756,2019,12,31,20,湖口,13.3,0.27,0.6,7.4,8.0,35.0,22.0,10.0,0.0,81.0,1.5,49.0,48.0,7.6,7.3
8757,2019,12,31,21,湖口,13.4,0.26,0.5,6.3,6.8,36.7,22.0,12.0,0.0,80.0,1.7,49.0,48.0,8.0,7.5
8758,2019,12,31,22,湖口,13.5,0.25,0.3,6.8,7.2,33.9,25.0,14.0,0.0,80.0,1.9,47.0,45.0,5.4,5.8


## Mission 3
<img style="float: left;" src="mission3.png" width="20%"><br>
<br>
請參照空品網即時 空氣品質指標(AQI) 計算公式求取逐時 AQI 指數值
並新設欄位Alert(0/1) 
<br>
註明 紫爆(AQI>200) 以上 的記錄 (Alert==1)

In [3]:
def cal_aqi(value, item):
    if(item == 'O3' and value > 300):
        item = 'O3_8h'
    for i in range(1,8):
        low = aqi.loc[item,'low{}'.format(i)]
        high = aqi.loc[item,'high{}'.format(i)]
        if (value>=low and value<=high):
            break
    aqi_h = aqi.loc['AQI','high{}'.format(i)]
    aqi_l = aqi.loc['AQI','low{}'.format(i)]
    ret_aqi=(((aqi_h-aqi_l) / (high-low)) *(value-low))+aqi_l
    return ret_aqi
    
aqi = pd.read_csv('aqi.csv')
aqi.set_index('item', inplace=True)
df2 = df_transform.copy()
df2.drop(['AMB_TEMP','NO','NOx','RAINFALL','RH','WD_HR','WIND_DIREC','WIND_SPEED','WS_HR'], axis=1, inplace=True)
df2['O3_8h'] = df2['O3'].rolling(8).mean()
for i in range(df2.shape[0]):
    aqi_list=[]
    for j in range(5,12):
        item = df2.columns[j]
        aqi_j = cal_aqi(df2.loc[i,item], item)
        aqi_list.append(aqi_j)
    df2.loc[i,'AQI'] = round(max(aqi_list),2)

df2['Alert']=df2['AQI'].apply(lambda x: 1 if x > 200 else 0)

display(df2)
df3 = df2[df2['Alert']==1]
print('有{}天發生紫爆'.format(df3.shape[0]))
display(df3)

,西元年,月,日,時,觀測站,CO,NO2,O3,PM10,PM2.5,SO2,O3_8h,AQI,Alert
0,2019,01,01,00,湖口,0.17,4.4,35.8,14.0,8.0,1.3,NaN,25.97,0
1,2019,01,01,01,湖口,0.17,4.6,36.0,10.0,9.0,1.2,NaN,29.22,0
2,2019,01,01,02,湖口,0.16,3.7,37.7,12.0,12.0,1.2,NaN,38.96,0
3,2019,01,01,03,湖口,0.16,3.4,39.1,12.0,10.0,1.2,NaN,32.47,0
4,2019,01,01,04,湖口,0.15,3.2,40.1,5.0,6.0,1.5,NaN,19.48,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,2019,12,31,19,湖口,0.28,6.8,35.7,22.0,13.0,1.4,38.7875,42.21,0
8756,2019,12,31,20,湖口,0.27,7.4,35.0,22.0,10.0,1.5,38.2250,35.39,0
8757,2019,12,31,21,湖口,0.26,6.3,36.7,22.0,12.0,1.7,37.8000,38.96,0
8758,2019,12,31,22,湖口,0.25,6.8,33.9,25.0,14.0,1.9,37.2250,45.45,0


有90天發生紫爆


,西元年,月,日,時,觀測站,CO,NO2,O3,PM10,PM2.5,SO2,O3_8h,AQI,Alert
62,2019,01,03,14,湖口,0.39,10.5,33.8,1000.0,4.0,2.2,31.8625,896.0,1
231,2019,01,10,15,湖口,0.26,7.2,39.9,1000.0,431.0,1.6,31.7000,896.0,1
422,2019,01,18,14,湖口,0.21,7.4,56.1,1000.0,8.0,1.9,47.6500,896.0,1
539,2019,01,23,11,湖口,0.39,11.2,49.7,1000.0,431.0,2.5,48.1875,896.0,1
589,2019,01,25,13,湖口,0.32,6.3,70.7,1000.0,431.0,2.3,54.8750,896.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8438,2019,12,18,14,湖口,0.45,9.0,34.5,1000.0,17.0,1.2,76.4875,896.0,1
8583,2019,12,24,15,湖口,0.22,6.2,44.9,1000.0,1000.0,1.8,29.9000,896.0,1
8673,2019,12,28,09,湖口,0.25,2.1,0.0,1000.0,1000.0,0.0,33.1375,896.0,1
8693,2019,12,29,05,湖口,0.34,22.3,1.1,1000.0,1000.0,1.5,3.2875,896.0,1


In [4]:
df2['O3_8h'] = round(df2['O3_8h'],2)
df2.to_csv('hukou_aqi_2019.csv')